# Équation de Poisson - Méthode des différences finies

## Définition du problème et des notations

L'objectif de ce projet est de résoudre l'équation de Poisson à 2D par la méthode des différences finies. Dans le domaine continue, en supposant le problème stationnaire et pour un domaine de l'espace tel que $x \in [0,L_x]$ et $y \in [0,L_y]$, cette équation prend la forme suivante :
$\frac{\partial^2 f}{\partial x^2}(x,y) + \frac{\partial^2 f}{\partial y^2}(x,y) = g(x,y)$ où g est fonction indépendante du temps. On rajoute à cette équation un jeu de conditions aux limites, on supposera lors de ce projet que les conditions aux limites imposées sont des conditions aux limites de Dirichlet (nous nous interesserons au cas du condensateur plan dans lequel les conditions aux limites sont de type Dirichlet) :
- $f(x,0) = \alpha^{Sud}(x)$,
- $f(x,L_x) = \alpha^{Nord}(x)$,
- $f(0,y) = \alpha^{Ouest}(y)$,
- $f(L_y,y) = \alpha^{Est}(y)$.

## Introduction de la méthode de résolution numérique : maillage de l'espace

Pour résoudre cette équation numériquement, l'idée va être de subdiviser l'espace continu en un ensemble de points discrets formant un maillage. Supposons que l'on prenne $(M_x+1)$ points selon la direction $x$ et $(M_y+1)$ points selon la direction $y$. Dès lors, le maillage de l'espace va-être constitué de $N = (M_x+1)(M_y+1)$ points tels que :
- 0 = $x_0$ < $x_1$ < ... < $x_{M_x} = L_x$,
- 0 = $y_0$ < $y_1$ < ... < $y_{M_y} = L_y$.

Ainsi, le problème revient à chercher l'expression de la fonction f en chacun des points du maillage, *ie* les N valeurs **nodales** de f notées $f(x_i,y_j) = f_{i,j}$. 
Par ailleurs, on suppose connu la valeur de g en N points du maillage (N valeurs nodales de g) : $g(x_i,y_j) = g_{i,j}$. Enfin, les conditions aux limites ci-dessus peuvent s'écrirent de la manière suivante dans le domaine du maillage :
- $\forall j \in [[0,M_y]]$, $f_{0,j} = \alpha^{Ouest}(y_j) = \alpha^{Ouest}_j$ et $f_{M_x,j} = \alpha^{Est}(y_j) = \alpha^{Est}_j$,
- $\forall i \in [[1,M_x-1]]$ (on évite de se répéter dans les coins --> $f_{0,0}$ et $f_{M_x,M_y}$ sont déjà décrits par le tiret ci-dessus donc on commence à 1 et on termine à $(M_x-1)$ pour ne pas avoir de doublon) $f_{i,0} = \alpha^{Sud}(x_i) = \alpha^{Sud}_i$ et $f_{i,M_y} = \alpha^{Nord}(x_i) = \alpha^{Nord}_i$.

Cela donne donc finalement $N_{limite} = 2M_x + 2M_y$ équations pour les termes de bord.
Interessons nous désormais à l'équation de Poisson dans le domaine du maillage. Considérons pour cela un maillage uniforme, *ie* $x_{i+1} - x_{i} = \delta x$ et $y_{j+1} - y_{j} = \delta y$ $\forall (i,j) \in [[0,M_x]]\times [[0,M_y]]$. On obtient alors facilement que $\frac{\partial^2 f}{\partial x^2}(x_i,y_j) = \frac{f_{i-1,j} + f_{i+1,j} - 2f_{i,j}}{\delta_x^2}$ et $\frac{\partial^2 f}{\partial y^2}(x_i,y_j) = \frac{f_{i,j-1} + f_{i,j+1} - 2f_{i,j}}{\delta_y^2}$. L'équation de Poisson discrète est donc donnée par $\frac{f_{i-1,j} + f_{i+1,j} - 2f_{i,j}}{\delta_x^2} + \frac{f_{i,j-1} + f_{i,j+1} - 2f_{i,j}}{\delta_y^2} = g_{i,j}$ $\forall (i,j) \in [[1,M_x-1]]\times [[1,M_y-1]]$ On obtient ainsi $N_{in} = (M_x-1)(M_y-1)$ équations.

## Première méthode : inversion classique avec np.linalg.solve

La première idée que l'on peut avoir consiste à mettre le système d'équations présenté ci-dessus sous la forme d'une équation matricielle unique de la forme $AF = G$, puis ensuite à utiliser la méthode  implémentée dans numpy, **np.linalg.solve**, pour en trouver la solution (cette fonction inverse simplement la matrice A pour calculer $F = A^{-1}G$). On obtiendrait ainsi la valeur de la fonction f en chacun des points du maillage, ces valeurs étant contenues dans la matrice $F$.

Pour appliquer cette démarche, il faut réfléchir à ce que doivent contenir les matrices $A$, $F$ et $G$. $F$ est un vecteur colonne de taille (N,1) puisqu'il doit contenir les valeurs de la fonction f en chacun des N points du maillage. Pour pouvoir écrire le système matricielle, il faut donc que la matrice A soit de taille (N,N) et que la matrice G soit un vecteur colonne de taille (N,1) contenant les valeurs de la fonction g en chacun des points du maillage. De manière à pouvoir arranger les coefficients dans la matrice A, il est nécessaire d'imposer l'ordre des éléments dans le vecteur F (le vecteur est représenté en ligne mais c'est bien un vecteur colonne) : $(f_{0,0},f_{0,1},...,f_{0,M_y}|f_{1,0},...,f_{1,M_y}|...|f_{M_x,0},...,f_{M_x,M_y})$. Le vecteur F est donc un vecteur de taille N, contenant N "sous vecteurs", chacun de taille $M_y+1$. Ce choix de numérotation a plusieurs conséquences :
- La matrice A est de taille **(N,N)** et est composée de $N\times N$ blocs, chacun de taille $(M_x+1,M_y+1)$.
- Les $(M_y+1)$ premiers et les $(M_y+1)$ derniers éléments du vecteur F correspondent respectivement aux valeurs de la fonction f sur les bords Ouest et Est du maillage. On connais donc déjà les $(M_y+1)$ premiers et derniers élements du vecteur G, respectivement $(\alpha^{Ouest}_0,...,\alpha^{Ouest}_{M_y})$ et $(\alpha^{Est}_0,...,\alpha^{Est}_{M_y})$. On connais également les premier (en haut à gauche) et dernier (en bas à droite) blocs de la matrice A : ce sont des blocs diagonaux portant des 1 sur leur diagonale.
- Les premiers et derniers termes de chacun des N-2 sous vecteurs restant composants le vecteur F (entre le deuxième et l'avant dernier sous vecteur, les premiers et derniers étant déjà traités dans le tiret ci-dessus) correspondent respectivement aux valeurs prisent par la fonction F sur les bords Sud et Nord du maillage. On peut donc déjà affirmer que le vecteur G va être de la forme suivante : $(\alpha^{Ouest}_0,...,\alpha^{Ouest}_{M_y}|\alpha^{Sud}_0,.?.,\alpha^{Nord}_0|...|\alpha^{Sud}_{M_x},.?.,\alpha^{Nord}_{M_x}|\alpha^{Est}_0,...,\alpha^{Est}_{M_y})$. Par ailleurs, on connait également les premiers et derniers termes de chacun des blocs diagonaux de la matrice A (entre le second et l'avant dernier bloc, les premier et dernier blocs étant déjà traités dans le tiret ci-dessus) : des 1. 

À partir d'ici, nous avons déjà rempli la matrice A et le vecteur G avec tous les élements concernant les termes de bords du maillage. La matrice A prend pour l'instant la forme suivante :

<img src="matrice_A.png" width="400" height="200">

On peut désormais chercher à remplir la matrice A avec les termes correspondant aux points intérieurs du maillage. Pour cela, il s'agit de reprendre l'expression de l'équation de Poisson établi dans la partie **"Introduction de la méthode de résolution numérique : maillage de l'espace"**, et de factoriser les différents termes : $\frac{f_{i+1,j} + f_{i-1,j}}{\delta x^2} + \frac{f_{i,j+1} + f_{i,j-1}}{\delta y^2} - 2f_{i,j}\left (\frac{1}{\delta x^2} + \frac{1}{\delta y^2} \right ) = g_{i,j} \Rightarrow \frac{f_{i+1,j} + f_{i-1,j}}{\delta x^2} + \frac{f_{i,j+1} + f_{i,j-1}}{\delta y^2} - 2f_{i,j}\left (\frac{1}{\delta x^2} + \frac{1}{\delta y^2} \right ) = g_{i,j} $. Posons alors $t_x = \frac{1}{\delta_x^2}$, $t_y = \frac{1}{\delta_y^2}$ et $t_{xy} = -\left (\frac{1}{\delta_x^2} + \frac{1}{\delta_y^2} \right )$. Compte tenu de l'ordre choisi pour les $f_{i,j}$ dans le vecteur F, il est assez simple désormais de déterminer les termes "intérieurs" de la matrice A. La matrice A prend alors la forme suivante :

<img src="matrice_A_p.png" width="400" height="200">

Par ailleurs, le vecteur G prend la forme $(\alpha^{Ouest}_0,...,\alpha^{Ouest}_{M_y}|\alpha^{Sud}_0,g_{1,1},...,g_{1,M_y},\alpha^{Nord}_0|...|\alpha^{Sud}_{M_x},g_{M_x,1},...,g_{M_x,M_y},\alpha^{Nord}_{M_x}|\alpha^{Est}_0,...,\alpha^{Est}_{M_y})$.
Ainsi, tout l'enjeu du code nommé **poisson_solve_classic** dans le fichier **Projet_equation_poisson** est d'initialiser la matrice A et le vecteur G de manière correcte. Pour ce faire, nous avons recours à un nouvel indice. En effet, les $f_{i,j}$ sont indexés par les indices 2D (i,j) mais ils se trouvent tous dans un grand vecteur colonne, ce qui nous permet d'imposer une nouvelle indexation 1D : $indice_{1D} = i\times(ny+1) + j$. Cela permet d'utiliser un seul indice pour se déplacer à la fois dans la matrice A et dans le vecteur G, ce point est plus détaillé dans les commentaires du code **poisson_solve_classic** au moment de l'utilisation de cet indice. Cette méthode sera appliquée par la suite au cas du potentiel électrostatique $V$ au sein d'un condensateur, qui vérifie l'équation de poisson $\Delta V = -\frac{\rho}{\varepsilon_0}$, (voir pour cela la partie **Application et comparaison des différentes méthodes**).

Nous avons vu que la matrice A est carrée, de taille $N\times N$. Supposons que l'on prenne pour le maillage $M_x = M_y = 65$ (ce qui n'est pas énorme, simplement correcte comme nous le verrons par la suite), on a alors N = 4356 ($= (M_x+1)(M_y+1)$), et A, qui est donc une matrice 4356$\times$4356, contient environ 19 millions d'éléments. Les méthodes directes telles que celle du Pivot de Gauss utilisées ici peuvent être exploitées mais sont peu efficaces et peuvent présenter des problèmes de stabilité pour les très grands systèmes (nous verrons par la suite que la matrice peut devenir numériquement singulière pour des valeurs de N trop importantes).
On peut cependant remarquer que A est une matrice creuse : chaque ligne de la matrice, correspondant à une équation, comporte seulement 5 éléments non nuls. Nous allons donc préférer utiliser des méthodes itératives : la méthode de Jacobi, la méthode de Gauss-Seidel ou la méthode de Gauss-Seidel adaptative, qui exploitent la relation de récurrence écrite ci-dessus et le caractère creux de la matrice des coefficients.

## Seconde méthode : méthode de Jacobi

La méthode de Jacobi est une méthode itérative de résolution d’un système d’équations linéaires. Partant d’une matrice $U_0$ comportant des valeurs initiales nulles, nous allons appliquer la relation de récurrence présentée plus haut pour calculer une suite de matrices $U_k$.
Pour comprendre comment cette méthode fonctionne, commençons par réécrire la relation de récurrence sous la forme suivante : $f_{i,j} = \left (\frac{f_{i+1,j} + f_{i-1,j}}{\delta x^2} + \frac{f_{i,j+1} + f_{i,j-1}}{\delta y^2} - g_{i,j} \right )\frac{1}{2}\frac{1}{\frac{1}{\delta x^2} + \frac{1}{\delta y^2}}$. C'est cette formule qui est implémentée dans le code intitulé **poisson_solve_jacobi** dans le fichier **Projet_equation_poisson**. Cependant, pour interpréter plus efficacement la formule précédente, on peut supposer, comme ce sera d'ailleurs toujours le cas dans les différents exemples que nous développerons par la suite, que les pas de maillage dans les directions $x$ et $y$ sont les mêmes : $\delta x = \delta y$. On pose par de plus $g_{i,j}' = \delta x^2g_{i,j}$. L'expression de $f_{i,j}$ peut alors se mettre sous la forme $f_{i,j} = \frac{f_{i+1,j} + f_{i-1,j} + f_{i,j+1} + f_{i,j-1}}{4} - \frac{g_{i,j}'}{4}$. Cette dernière expression permet de clairement visualiser $f_{i,j}$ comme la somme de la valeur moyenne des "cases" voisines de $(i,j)$ (premier terme) et d'un terme de perturbation (le second terme).
L'idée de la méthode itérative est alors assez simple et peut se résumer par les trois étapes suivantes :
- $\textbf{Initialisation}$ : on commence par initialiser une matrice U de taille $(M_x,M_y)$ (et non $(M_x+1,M_y+1)$ comme la taille des blocs de la matrice A dans la partie précédente --> changement de convention à noter) telle que  $\forall (i,j) \in \partial\Omega, U(i,j) = CL(i,j)$ et $\forall (i,j) \in \Omega /\partial\Omega, U(i,j) = 0$, autrement dit les bords de la matrice sont remplis par les valeurs de f sur la frontière du maillage (les différentes conditions aux limites $\alpha$ définies dans la partie **Définition du problème et des notations**) et les termes "intérieurs" sont fixés à 0.
- $\textbf{Itération}$ : Une itération consiste à effectuer, pour tout point (i,j) n'appartenant pas à un bord, le calcul d’une nouvelle valeur de f selon l’équation de récurrence définie ci-dessus. Ainsi f à l’itération k+1 est obtenue en fonction de f à l’itération k via la formule $f_{i,j}^{k+1} = \left (\frac{f_{i+1,j}^k + f_{i-1,j}^k}{\delta x^2} + \frac{f_{i,j+1}^k + f_{i,j-1}^k}{\delta y^2} - g_{i,j} \right )\frac{1}{2}\frac{1}{\frac{1}{\delta x^2} + \frac{1}{\delta y^2}}$. Dans le code, cela nécessite de faire une copie de la matrice U contenant les $f_{i,j}$ à chaque itération, en utilisant la fonction *U_old = U.copie*.
- $\textbf{Terminaison}$ : On stoppe la simulation lorsque les itérations ne font quasiment plus évoluer les valeurs contenues dans la matrice U, ou autrement dit lorsque le terme de perturbation ne modifie plus significativement le terme de moyenne. On définit pour cela l’écart $\varepsilon_k = max_{i,j}|U^{k+1} - U^{k}|$, correspondant à l'écart maximale existant entre les coefficients de la matrice U à l'itération (k+1) et les coefficients de la matrice U à l'itération k (que l'on a conservée en en faisant une copie). L'idée que nous avons implémentée dans notre programme consiste à dire que l'on sort de la boucle lorsque $\varepsilon_k$ devient inférieur à une valeur *tol* fixée en entrée de la fonction, le critère de convergence. Cette étape de test est placée dans une boucle while (voir code **poisson_solve_jacobi**). Notons que nous avons rajouté une troisième boucle (en plus de celles sur i et j) pour "compter" le nombre d'itérations réalisées. En faisant cela, nous cherchons à éviter de faire tourner le code trop longtemps : on limite donc cette troisième boucle à un nombre maximal d'itérations défini en entrée de la fonction (*max_iter = 1000* par défaut), au dela duquel on sort de la boucle et on renvoie la matrice calculée au bout de ce nombre maximal d'itérations. Une idée intéressante aurait été de remplacer le critère d'arrêt utilisé par $\varepsilon_k = \frac{||U^{k+1} - U^{k}||_F}{\sqrt{M_xM_y}}$ où $||.||_F$ est la norme de Frobenius sur $\mathrm{M}_{M_x,M_y}(K)$, qui dérive du produit scalaire ou hermitien standard. L'avantage de ce critère de convergence est que le facteur $\frac{1}{\sqrt{M_xM_y}}$ permet de considérer la valeur de $\varepsilon_k$ comme une erreur moyenne et donc de rendre cette valeur assez peu sensible au choix du pas, ce qui permet d'appliquer la méthode de Jacobi avec le même seuil $\textit{tol}$ pour différentes valeurs de $(M_x,M_y)$.

Pour un critère de convergence *tol* fixé, si l'on suppose que $M_x = M_y = N$ (*attention, ce n'est pllus le même N que dans la Méthode classique où on avait $N = (M_x+1)(M_y+1)$)*, la méthode de Jacobi converge en $\mathcal{O}(N^2)$ itérations. Comme chaque itération requiert le parcours de la grille et donc $\mathcal{O}(N^2)$ opérations, la complexité globale de cette méthode est en $\mathcal{O}(N^4)$. Bien que meilleur que la méthode d'élimination de Gauss-Jordan, la complexité de la méthode de Jacobi ne permet pas de dépasser des domaines de taille $N$ de l’ordre de 100 en gardant des temps de simulations raisonnables (voir la partie **Application et comparaison des méthodes**). Cependant, il existe des améliorations qui permettent de diminuer la complexité en $\mathcal{O}(N^3)$, tout en conservant un algorithme simple : les méthodes de Gauss-Seidel et de Gauss-Seidel adaptative.

## Amélioration de la méthode de Jacobi : méthode de Gauss-Seidel

La méthode de Jacobi nécessite, à chaque itération, le calcul d'une nouvelle matrice. On lui préfère généralement la méthode de Gauss-Seidel, un peu plus efficace et ne nécessitant qu'une seule matrice. Cette amélioration de la méthode de Jacobi consiste à modifier la matrice en ligne, $\textit{ie}$ sans créer une nouvelle matrice. La formule de récurrence précédement utilisée est remplacée par l'expression suivante : $f_{i,j}^{k+1} = \left (\frac{f_{i+1,j}^k + f_{i-1,j}^{k+1}}{\delta x^2} + \frac{f_{i,j+1}^k + f_{i,j-1}^{k+1}}{\delta y^2} - g_{i,j} \right )\frac{1}{2}\frac{1}{\frac{1}{\delta x^2} + \frac{1}{\delta y^2}}$, où les valeurs de f en **(i−1, j)** et **(i, j−1)** sont celles de l’itération en cours k+1. On va donc parcourir la grille avec i et j croissants, afin que ces valeurs aient été déjà actualisées lors de cette itération (autrement dit,  au moment de calculer $f_{i, j}^{k+1}$, on a déjà calculé $f_{i−1,j}^{k+1}$ et $f_{i,j-1}^{k+1}$, mais pas encore $f_{i+1,j}^{k+1}$ ni $f_{i,j+1}^{k+1}$). Comme les valeurs courantes ont déjà été mises à jour, cela permet d’améliorer la vitesse de convergence (gain d’un facteur 2). On peut démontrer que ce schéma converge un peu plus rapidement que le schéma de Jacobi. Cependant, bien que ne nécessitant plus de copie de matrice pour le calcul récurssif, le test d’arrêt restant le même que dans la méthode de Jacobi, il faut quand même conserver une copie du tableau précédent pour calculer l’écart entre le nouveau tableau et l’ancien tableau (critère d'arrêt) et par conséquent cette méthode ne permet pas un gain très important, elle converge également en $\mathcal{O}(N^4)$ ($M_x = M_y = N$). Par contre, la méthode de sur-relaxation permet d'accélérer considérablement la convergence de la méthode de Gauss-Seidel.

## Amélioration de la méthode de Gauss-Seidel : méthode de Gauss-Seidel adaptative (sur-relaxation)

La méthode de relaxation de Gauss-Seidel adaptative consiste à modifier la méthode de Gauss-Seidel de manière à ce que la nouvelle valeur renvoyée après chaque itération soit obtenue comme une moyenne pondérée entre l’estimation donnée par Gauss-Seidel et l’ancienne valeur de f au point considéré. Du point de vu mathématique, la méthode de sur-relaxation consiste à exprimer $f_{i,j}^{k+1}$ comme une combinaison convexe de $f_{i,j}^{k}$ et de la valeur donnée par le schéma de Gauss-Seidel, ce qui se traduit par l'expression suivante : $f_{i,j}^{k+1} = (1-\omega)f_{i,j}^{k} + \omega\left (\frac{f_{i+1,j}^k + f_{i-1,j}^{k+1}}{\delta x^2} + \frac{f_{i,j+1}^k + f_{i,j-1}^{k+1}}{\delta y^2} - g_{i,j} \right )\frac{1}{2}\frac{1}{\frac{1}{\delta x^2} + \frac{1}{\delta y^2}}$, où le facteur de pondération $\omega$ est appelé "facteur de relaxation". Il est possible de démontrer plusieurs propriétés que nous admettrons ici :
- cette méthode converge pour 0 < $\omega$ < 2,
- cette méthode converge plus rapidement que la méthode de Gauss-Seidel pour 1 < $\omega$ < 2 (on parle de sur-relaxation tandis que si 0 < $\omega$ < 1, on parle de sous-relaxation),
- il existe une valeur optimale de $\omega$ pour laquelle on obtient une valeur approchée du résultat à *tol* près en moyenne, avec un nombre d’itérations en $\mathcal{O}(N)$. Pour le problème de Poisson considéré ici, on peut montrer que (si $M_x = M_y = N$) cette valeur optimale de $\omega$ est donnée par $\omega_{opt} = \frac{2}{1+\frac{\pi}{N}}$. Notons que ce résultat est valide pour l’équation de Laplace ou de Poisson, avec conditions aux bords de Dirichlet ou de Neumann. On remarque que lorsque N augmente, $\omega_{opt}$ se rapproche de 2. Nous avons implémenter cette formule dans le code intitulé **poisson_solve_gauss_seidel_adapt** : l'utilisateur a alors le choix de laisser l'algorithme caculer la valeur optimale en utilisant cette formule (option *omega='c'*) ou de fixer la valeur lui même (option *omega=entier au choix*).

Intéressons nous à la complexité de cette méthode en supposant que $M_x = M_y = N$. À chaque itération, il faut calculer les $N^2 = \mathcal{O}(N^2)$ coefficients du tableau U, ces calculs ayant à peu près tous le même coût. Pour $\omega = \omega_{opt}$, on sait que le nombre d’itérations est en $\mathcal{O}(N)$, donc le coût total du schéma par sur-relaxation est en $\mathcal{O}(N^3)$ : c'est bien ce que l'on souhaitait !

## Application et comparaison des différentes méthodes

### Application des méthodes

Pour appliquer les quatre méthodes décrites et implémentées, nous allons nous appuyer sur l'exemple du potentiel électrostatique généré par un condensateur plan formé de deux armatures métalliques soumisent respectivement au potentiel $-V_0$ et $+V_0$. Ces deux armatures sont caractérisées par des densités surfaciques de charges $\rho$. Le potentiel électrostatique $V(x,y)$ vérifie alors une équation de Poisson de la forme $\frac{\partial^2V}{\partial x^2} + \frac{\partial^2V}{\partial y^2} = \frac{\rho}{\varepsilon_0}$, avec $\varepsilon_0$ la permititivité diélectrique du vide. 

Pour l'implémentation du problème, nous avons considéré les élements suivants :
- Le domaine de maillage que l'on considère est un "carré" de côté $L_x = L_y = 1$,
- Dans un premier temps (pour pouvoir vérifier que toutes les méthodes fonctionnent correctement), on choisi $M_x = M_y = 65$ subdivisions du maillage dans chacune des directions $x$ et $y$, 
- On construit la matrice g de façon à placer les plaques en y = 0.4 et y = 0.6, chacune des plaques s'étendant selon x entre 0.25 et 0.75 : on place donc des 1 sur toute la ligne en *int(0.4*N)* et des -1 sur toute la ligne *int(0.6*N)* plaques respectivement chargées + et -) --> dans la méthode classique, cette matrice est transformée en vecteur colonne dans le code,
- Les conditions aux limites sont toutes prises égale à 0 : - $\alpha^{Sud}(x) = \alpha^{Nord}(x) \alpha^{Ouest}(y) = \alpha^{Est}(y) = 0$. Concrétement, on impose donc un potentiel nul sur le cadre du domaine, ce qui signifie que le condensateur est en réalité dans une boîte dont le bord est au potentiel nul. Si la boîte est assez grande, comme le potentiel du condensateur tend vers 0 à l’infini, celle-ci n’aura qu’une influence négligeable.
- Pour les méthodes de Jacobi, Gauss-Seidel et Gauss-Seidel adaptative, on choisit dans un premier temps un facteur de convergence $tol$ égal à $10^{-6}$ (par défaut dans les paramètres d'entrée des trois fonctions).

Avec les paramètres ainsi définis, l'application de chacune des quatres méthodes nous permet d'obtenir une matrice contenant les valeurs du potentiel électrostatique en chacun des points de l'espace, puis de tracer les lignes équipotentiels en utilisant la fonction *plt.contour*. Si on représente 100 lignes équipotentiel pour chacune des méthodes, on obtient les figures ci-dessous : 

<img src="equi_pot_2.png" width="500" height="300">

On constate que les quatre méthodes aboutissent à peu-prêt au même résultat, on peut donc s'y fier. Cependant les méthodes de Jacobi et de Gauss-Seidel donne des résultats légérements différents des deux autres : la précision *tol* ou la taille du maillage ne sont sans doute pas encore suffisant pour s'affranchir de ces erreurs comme nous le verrons par la suite (ces deux méthodes demandent une précisions plus grande ou un maillage plus important pour donner des résultats corrects). De plus, si on augmente la taille du maillage à 5, on constate que les lignes équi-V ne sont pas modifiées, ce qui montre bien que la boîte est assez grande pour considérer son influence comme négligeable.
Par ailleurs, connaissant le potentiel, les composantes du champ électrique peuvent être calculées en utilisant la formule  $\vec{E}(x,y) = -\vec{grad}$ $V(x,y)$. Pour réaliser ce calcul, on pourrait simplement utilisé la méthode d'Euler, mais il faut se méfier des termes de bord lors de l'indexation. Nous préferons donc utiliser la fonction gradient directement implémentée dans numpy et utilisant un schéma numérique plus précis que le schéma d’Euler et gérant directement ces effets de bord : *Ex, Ey = np.gradient(potentiel)*. Avec cette fonction, Ex et Ey sont directement des matrices de mêmes dimensions que le potentiel. On peut donc venir tracer ces lignes de champs avec la fonction *plt.streamplot*. On obtient ainsi les figures suivantes :

<img src="lignes_champ.png" width="500" height="300">

On constate que les lignes de champ sont bien orthogonales aux lignes équipotentiel, c'est ce que l'on voulait. Par ailleurs, on retrouve des discontinuités du champ au passage des plans chargés en surface, ce qui est cohérent avec la relation de passage. Enfin, les lignes de champ à l'intérieur du condensateur sont bien des droites parallèles, sauf quand on se rapproche des bords : les effets de bord (condensateur fini dans le plan (x,y), infini selon z puisque l'on est à 2D) ont un effet notable sur le champ à l’intérieur du condensateur sur une distance de l’ordre de l’épaisseur interarmatures (0.2 u.a.).

Pour pouvoir avoir accès à ces simulations, il suffit d'entrer **1** lorsque la question *"Tracé des courbes équipotentiels et des lignes de champ (entrer 1) ou comparaison des méthodes en temps et en consommation de mémoire (entrer 2) ou influence de la précision et nombre d'itérations (entrer 3)"* est posée.

### Comparaison des méthodes 

#### Comparaison en terme de durée d'exécution

Nous allons maintenant pouvoir chercher à comparer les méthodes. Nous pouvons tout d'abord nous intéresser aux durées d'exécution de chacune des méthodes en fonction du nombre de points dans le maillage. Pour cela, on utilise la fonction *time.time*. Si l'on fait varier N entre 0 et 120, on obtient les courbes suivantes :

<img src="time_versus_N.png" width="500" height="300">

Plusieurs commentaires peuvent être fait :
- Pour de petites valeurs de N (jusqu'à N = 30), les quatres méthodes ont des temps d'exécution totalement comparables. Cependant, pour des subdivisions si grossières, le résultat renvoyé n'est pas précis du tout, comme nous le verrons par la suite.
- Entre N = 30 et N = 70, on constate que la méthode de Gauss-Seidel adaptée est la plus rapide, suivi de la méthode classique, puis de la méthode de Gauss-Seidel et enfin de la méthode de Jacobi. C'est assez logique : on a montré que pour une valeur optimale du facteur de relaxation, la complexité de la méthode de Gauss-Seidel adaptée était en $\mathcal{O}(N^3)$, tandis que les autres sont au moins en $\mathcal{O}(N^4)$. Par ailleurs, la méthode classique est encore assez efficace jusqu'à N = 70, les matrices rentrant en jeu n'étant pas encore de taille trop importante (bien que vers N = 60-70, les matrices mis en jeu commencent à être réellement conséquentes) pour être vu comme singulière par $\textit{np.linalg.solve}$.
- À partir de N = 70, la durée d'exécution de la méthode classique explose et dépasse largement celle des autres méthodes : on a atteint les limites des capacités d'inversion de matrice de la fonction *np.linalg.solve* : c'est ce que nous avions prévu précédement et c'est pour cette raison que nous avons développé des méthodes itératives qui reste assez stable quand N dépasse 70.
- Aux alentours de N = 100, la durée d'exécution de la méthode de Jacobi diminue et passe en dessous de celle des deux autres méthodes (voir figure ci-dessous) et il semble que la méthode de Gauss-Seidel suive la même tendance (voir figure ci-dessous). On peut expliquer cela assez facilement : le nombre d'itérations maximales - 1000 - est atteint par la méthode de Jacobi, tandis que celui de la méthode de Gauss-Seidel continue d'augmenter. Il faut donc augmenter le nombre maximum d'itérations possibles pour voir ce phénomène disparaitre. Cependant, cela n'explique pas pourquoi le temps d'exécution **diminue**, il devrait en effet rester constant. Étrangement, ce phénomène n'est plus observé quand on diminue la tolérance n de la méthode à $10^{-7}$.

<img src="3_methodes.png" width="500" height="300">

Par ailleurs, nous avions affirmé que la complexité de l'algorithme de Gauss-Seidel adapté était en $\mathcal{O}(N^3)$. On peut le vérifier en traçant le logarithme du temps d'exécution de la méthode de Gauss-Seidel adaptée en fonction de log(N) et en effectuant une régression linéaire. On doit normalement trouver un coefficient directeur voisin de 3. Le résultat est présenté sur la figure ci-dessous.

<img src="Gaus_Seidel_adapt.png" width="500" height="300">

Le coefficient directeur du fit linéaire vaut 2.81 : c'est donc totalement cohérent avec une complexité en $\mathcal{O}(N^3)$ !

Pour pouvoir avoir accès à ces simulations, il suffit d'entrer **2** lorsque la question *"Tracé des courbes équipotentiels et des lignes de champ (entrer 1) ou comparaison des méthodes en temps et en consommation de mémoire (entrer 2) ou influence de la précision et nombre d'itérations (entrer 3)"* est posée.

#### Comparaison en terme d'occupation de la mémoire

On peut maintenant s'intéresser à la place en mémoire occupée par chacune des méthodes. Pour mesurer cela, nous avons utilisé la fonction *tracemalloc* (voir dans le code pour son fonctionnement). On obtient de cette manière l'évolution de la mémoire en fonction de la dimension du maillage utilisé sur les courbes ci dessous.

<img src="memoire_1.png" width="500" height="300">

On observe, comme nous l'avions prévu, que l'occupation mémoire de la méthode classique est bien plus importante que celle des trois autres méthodes. C'est assez logique au vu des tailles de matrices mis en jeu dans la méthode classique : $M_x^2\times M_y^2$ pour la méthode classique contre $M_x\times M_y$ pour les trois autres méthodes. Regardons maintenant ce qu'il en est pour les trois autres méthodes en retirant la courbe correspondant à la méthode classique, qui écrase le tout :

<img src="memoire_2.png" width="500" height="300">

On constate que la consommation de mémoire est la même pour chacune des trois méthodes. C'est tout à fait logique. En effet, le principe de fonctionnement de ces trois méthodes est assez similaire, et les objets manipulés sont de même taille : dans les trois cas, les matrices sont de taille $N\times N$ et dans les trois cas, deux matrices sont stockées en mémoire, la matrice de l'itération actuelle et de l'itération précédente, même si dans le cas des méthodes de Gauss-Seidel et de Gauss-Seidel adaptée, l'ancienne matrice n'est pas directement utilisée dans le calcul mais seulement dans le critère d'arrêt. Dès lors, il est logique que l'occupation mémoire soit identique pour chacune des trois méthodes. Notons enfin que pour les quatres méthodes, il semble que l'évolution de l'occupation mémoire se fasse de manière exponentielle par rapport à la taille du maillage.

Pour pouvoir avoir accès à ces simulations, il suffit d'entrer **2** lorsque la question *"Tracé des courbes équipotentiels et des lignes de champ (entrer 1) ou comparaison des méthodes en temps et en consommation de mémoire (entrer 2) ou influence de la précision et nombre d'itérations (entrer 3)"* est posée.

#### Comparaison en terme d'influence du critère de convergence, de nombre d'itérations et de précision

On peut enfin s'intéresser à l'influence du critère de convergence ($\textit{tol}$) sur la qualité de la solution et le nombre d'itérations nécessaires pour parvenir à cette solution. Pour cela, on trace la solution obtenue avec les trois méthodes (Jacobi, Gauss-Seidel et Gauss-Seidel adaptative) pour quatre valeurs de critère de convergence : $10^{-5}$, $10^{-6}$, $10^{-7}$, $10^{-8}$ et on regarde le nombres d'itérations nécessaires pour obtenir chacune de ces solutions. On obtient les élements présentés ci-dessous avec un maillage de 50$\times$50 points.

*Remarque* : on débride pour cette partie le nombre maximal d'itérations que l'on peut réaliser en le plaçant à 10000 (*max_iter = 10000*), afin de ne pas être limité dans la comparaison des méthodes (notamment pour la tolérance $10^{-8}$, la méthode de Jacobi dépasse les 1000 itérations (1300 environ)).

<img src="number_it.png" width="500" height="300">
<img src="jacobi.png" width="500" height="300">
<img src="gauss_seidel.png" width="500" height="300">
<img src="gauss_seidel_adapt.png" width="500" height="300">

On constate, comme nous l'avions déjà remarqué lors de l'étude temporelle, que le nombre d'itérations dans la méthode de Gauss-Seidel adaptée est largement inférieur à celui des deux autres méthodes : à peine une centaine d’itérations pour la méthode adaptative, 500 pour la méthode de Gauss-Seidel et plus du double pour la méthode de Jacobi. 
Par ailleurs, avec la tolérance $10^{-5}$, on obtient un résultat tout à fait satisfaisant avec la méthode adaptative et des résultats abérrant avec les deux autres méthodes. Par conséquent, la méthode de Gauss-Seidel adaptative peut aller encore plus vite que les autres puisqu'elle demande une précision moins importante. En effet, dans l’idéal, on souhaiterait prendre un critère *tol* le plus petit possible, mais cela impliquerait des temps de calculs immenses, et il faut donc trouver un compromis entre rapidité et précision. Le critère est donc que *tol* est assez petit si les résultats de la simulation n’en dépendent quasiment plus. Dès lors, pour la méthode de Gauss-Seidel adaptée, une tolérance de $10^{-6}$ est largement suffisante, tandis que pour les deux autres méthodes, une tolérance d'au moins $10^{-7}$ est nécessaire, ce qui explique que le résultat présenté dans la partie **Application des méthodes** ne soit pas totalement correct pour ces deux méthodes.

## Conclusion

Pour conclure, nous avons mis en oeuvre ici des méthodes itératives visant à contourner les difficultés d'inversion numérique de systèmes linéaires rencontrer lorsque les matrices ont des tailles trop importante. Notons que nous n'avons pas implémenté des problèmes avec conditions de Neumann (nous n'en n'avons pas besoin pour le condensateur). Cependant, dans le domaine de la thermodynamique, ces conditions sont souvent rencontrées : barre calorifugée sur les côtés, ailette de refroidissement, pont thermique dans un bâtiment...
Enfin, notons que d’autres méthodes d’optimisation sont possibles, comme l'accélération de Chebyshev qui change la valeur du facteur de relaxation au cours des itérations ou le schéma multigrilles qui utilise des grilles de plus en plus fines. Cependant, ces méthodes sont significativement plus complexes à mettre en place.

